In [70]:
from tokenizers import Tokenizer, models, trainers, pre_tokenizers

tokenizer = Tokenizer(models.BPE(unk_token="[UNK]"))

In [71]:
tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel(add_prefix_space=False)
tokenizer.pre_tokenizer.pre_tokenize_str("Let's test\npre-tokenization!")

[('Let', (0, 3)),
 ("'s", (3, 5)),
 ('Ġtest', (5, 10)),
 ('Ċ', (10, 11)),
 ('pre', (11, 14)),
 ('-', (14, 15)),
 ('tokenization', (15, 27)),
 ('!', (27, 28))]

In [72]:
special_tokens=["<|endoftext|>","[UNK]"]
trainer = trainers.BpeTrainer(
    vocab_size=8000,
    special_tokens=special_tokens,
    show_progress=True,
    min_frequency=2,
)

In [73]:
from datasets import load_dataset, interleave_datasets

ds_train = load_dataset("HuggingFaceH4/no_robots", split="train", streaming=True)
ds_test = load_dataset("HuggingFaceH4/no_robots", split="test", streaming=True)

full_dataset=interleave_datasets(
    [ds_train, ds_test],
    probabilities=[0.95, 0.05],          # optional: weight toward train
    seed=42
)

full_dataset.select_columns(["messages"])

print(next(iter(full_dataset)))

{'prompt': 'Please summarize the goals for scientists in this text:\n\nWithin three days, the intertwined cup nest of grasses was complete, featuring a canopy of overhanging grasses to conceal it. And decades later, it served as Rinkert’s portal to the past inside the California Academy of Sciences. Information gleaned from such nests, woven long ago from species in plant communities called transitional habitat, could help restore the shoreline in the future. Transitional habitat has nearly disappeared from the San Francisco Bay, and scientists need a clearer picture of its original species composition—which was never properly documented. With that insight, conservation research groups like the San Francisco Bay Bird Observatory can help guide best practices when restoring the native habitat that has long served as critical refuge for imperiled birds and animals as adjacent marshes flood more with rising sea levels. “We can’t ask restoration ecologists to plant nonnative species or to 

In [74]:
from typing import Iterator


def plain_text_iterator(dataset) -> Iterator[str]:
    """
    Streams all text messages from the dataset.
    """
    for example in dataset:
        for msg in example["messages"]:
            yield msg["content"]

texts=plain_text_iterator(full_dataset)
print(next(texts))

Please summarize the goals for scientists in this text:

Within three days, the intertwined cup nest of grasses was complete, featuring a canopy of overhanging grasses to conceal it. And decades later, it served as Rinkert’s portal to the past inside the California Academy of Sciences. Information gleaned from such nests, woven long ago from species in plant communities called transitional habitat, could help restore the shoreline in the future. Transitional habitat has nearly disappeared from the San Francisco Bay, and scientists need a clearer picture of its original species composition—which was never properly documented. With that insight, conservation research groups like the San Francisco Bay Bird Observatory can help guide best practices when restoring the native habitat that has long served as critical refuge for imperiled birds and animals as adjacent marshes flood more with rising sea levels. “We can’t ask restoration ecologists to plant nonnative species or to just take thei

In [75]:
tokenizer.train_from_iterator(texts, trainer)
file_name= "./tokenizer.json"


In [76]:
from tokenizers import decoders, processors

tokenizer.post_processor = processors.ByteLevel(trim_offsets=False)
tokenizer.decoder = decoders.ByteLevel()

In [77]:
tokenizer.save(file_name)

In [78]:
# Load your trained tokenizer
tokenizer = Tokenizer.from_file(file_name)

text = "To be, or not to be:\n\nThat is \nthe question."

encoding = tokenizer.encode(text)
print(encoding.tokens)
decoding=tokenizer.decode(encoding.ids)
print(decoding)

['To', 'Ġbe', ',', 'Ġor', 'Ġnot', 'Ġto', 'Ġbe', ':', 'Ċ', 'Ċ', 'That', 'Ġis', 'Ġ', 'Ċ', 'the', 'Ġquestion', '.']
To be, or not to be:

That is 
the question.
